<a href="https://colab.research.google.com/github/Bharathi3009/Talent-Opportunity-Index/blob/main/TOI_ML_%26_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# 1. Load data
df = pd.read_csv('AI_Resume_Screening.csv')  # replace with actual CSV filename
print(df.columns) # Add this line to check columns

# 2. Handle missing values (fill or drop) for relevant columns
df['Experience (Years)'] = df['Experience (Years)'].fillna(df['Experience (Years)'].mean())
df['Education'] = df['Education'].fillna('Bachelor')
df['Skills'] = df['Skills'].fillna('')

Index(['Resume_ID', 'Name', 'Skills', 'Experience (Years)', 'Education',
       'Certifications', 'Job Role', 'Recruiter Decision',
       'Salary Expectation ($)', 'Projects Count', 'AI Score (0-100)'],
      dtype='object')


In [30]:
from sklearn.preprocessing import StandardScaler

# 2. Handle missing values (fill or drop)
df['Experience (Years)'] = df['Experience (Years)'].fillna(df['Experience (Years)'].mean())
df['Education'] = df['Education'].fillna('Bachelor')
df['Skills'] = df['Skills'].fillna('')

# Normalize numeric features (e.g. Experience)
scaler = StandardScaler()
df['experience_norm'] = scaler.fit_transform(df[['Experience (Years)']])

# 3. Feature engineering: count number of skills (assuming comma-separated list)
df['num_skills'] = df['Skills'].apply(lambda x: 0 if x.strip()=='' else len(x.split(',')))
df.drop(columns=['Skills'], inplace=True)

# 4. Encode education level to numeric
edu_map = {'Bachelor': 0, 'Master': 1, 'PhD': 2}
df['education_code'] = df['Education'].map(edu_map)
df.drop(columns=['Education'], inplace=True)

In [31]:
import numpy as np
# 5. Prepare feature matrix X and label y

# Fill missing values in education_code before preparing X
df['education_code'] = df['education_code'].fillna(df['education_code'].mode()[0])

# 6. Prepare feature matrix X and label y
print(df.columns) # Add this line to check columns
print(df['Recruiter Decision'].dtype) # Check data type
print(df['Recruiter Decision'].unique()) # Re-check unique values
print(df['Recruiter Decision'].apply(lambda x: type(x)).unique()) # Check types of individual elements
X = df[['experience_norm', 'num_skills', 'education_code']]
df['Hire'] = np.where(df['Recruiter Decision'] == 'Hire', 1, 0)
y = df['Hire']

Index(['Resume_ID', 'Name', 'Experience (Years)', 'Certifications', 'Job Role',
       'Recruiter Decision', 'Salary Expectation ($)', 'Projects Count',
       'AI Score (0-100)', 'experience_norm', 'num_skills', 'education_code'],
      dtype='object')
object
['Hire' 'Reject']
[<class 'str'>]


In [32]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [33]:
# Train models
rf = RandomForestClassifier(n_estimators=100, random_state=42)
lr = LogisticRegression()
rf.fit(X_train, y_train)
lr.fit(X_train, y_train)

# Predict on test set
y_pred_rf = rf.predict(X_test)
y_pred_lr = lr.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:,1]
y_proba_lr = lr.predict_proba(X_test)[:,1]

In [34]:
# Evaluate metrics
print("Random Forest metrics:")
print("  Accuracy  =", accuracy_score(y_test, y_pred_rf))
print("  Precision =", precision_score(y_test, y_pred_rf))
print("  Recall    =", recall_score(y_test, y_pred_rf))
print("  ROC-AUC   =", roc_auc_score(y_test, y_proba_rf))
print()
print("Logistic Regression metrics:")
print("  Accuracy  =", accuracy_score(y_test, y_pred_lr))
print("  Precision =", precision_score(y_test, y_pred_lr))
print("  Recall    =", recall_score(y_test, y_pred_lr))
print("  ROC-AUC   =", roc_auc_score(y_test, y_proba_lr))

# Compute TOI scores (probability of hire) for each candidate (using logistic model)
toi_scores = lr.predict_proba(X)[:,1]
print("\nTOI (hire probability) for first 5 candidates:", toi_scores[:5])

Random Forest metrics:
  Accuracy  = 0.84
  Precision = 0.8700787401574803
  Recall    = 0.9364406779661016
  ROC-AUC   = 0.9214115466101694

Logistic Regression metrics:
  Accuracy  = 0.8433333333333334
  Precision = 0.8888888888888888
  Recall    = 0.9152542372881356
  ROC-AUC   = 0.9210805084745763

TOI (hire probability) for first 5 candidates: [0.99965529 0.99976913 0.42895407 0.99461965 0.92177726]


In [35]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Build simple MLP model
mlp = Sequential([
    Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])
mlp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train MLP
mlp.fit(X_train, y_train, epochs=20, batch_size=16, verbose=0)

# Predict and evaluate
y_proba_nn = mlp.predict(X_test).ravel()
y_pred_nn = (y_proba_nn >= 0.5).astype(int)

print("\nNeural Network metrics:")
print("  Accuracy  =", accuracy_score(y_test, y_pred_nn))
print("  Precision =", precision_score(y_test, y_pred_nn))
print("  Recall    =", recall_score(y_test, y_pred_nn))
print("  ROC-AUC   =", roc_auc_score(y_test, y_proba_nn))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 

Neural Network metrics:
  Accuracy  = 0.8433333333333334
  Precision = 0.8888888888888888
  Recall    = 0.9152542372881356
  ROC-AUC   = 0.920948093220339
